In [1]:
# ============================================================================
# CÉLULA 1: Importações e Configurações
# ============================================================================
import d3rlpy
import numpy as np
import pandas as pd
import joblib
import torch
import os
import shutil
import json
from d3rlpy.algos import CQLConfig
from d3rlpy.models import QRQFunctionFactory
from d3rlpy.dataset import ReplayBuffer, FIFOBuffer
from d3rlpy.metrics import AverageValueEstimationEvaluator
from sklearn.model_selection import train_test_split

# Configuração de sementes para reprodutibilidade
d3rlpy.seed(42)
np.random.seed(42)

print(f"d3rlpy version: {d3rlpy.__version__}")
print(f"Torch version: {torch.__version__}")
print(f"GPU disponível? {torch.cuda.is_available()}")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\MDF\Downloads\locac-main\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


d3rlpy version: 2.8.1
Torch version: 2.9.1+cpu
GPU disponível? False


In [2]:
# ============================================================================
# CÉLULA 2: Carregar Artefatos Gerados (Buffer e Scalers de Assinatura)
# ============================================================================
BUFFER_PATH = "rl_assinatura_buffer.h5" # Arquivo gerado pelo Bloco 4 do Generator_NEW

print(f"\n[1/5] Carregando ReplayBuffer de Assinatura de '{BUFFER_PATH}' ...")

# Carrega o buffer específico de assinatura
with open(BUFFER_PATH, "rb") as f:
    dataset = ReplayBuffer.load(f, FIFOBuffer(limit=100000))

print(f"✓ Buffer de Assinatura carregado com sucesso.")
print(f"  # Transições totais: {dataset.transition_count}")

# Carrega os Scalers ESPECÍFICOS de assinatura gerados pelo Generator_NEW
print("\n[2/5] Carregando Scalers de Assinatura...")
try:
    # Nota: O estado base usa o scaler comum, mas memória é separada
    scaler_estado_base = joblib.load("scaler_estado.joblib") 
    scaler_memoria = joblib.load("scaler_assinatura_memoria.joblib")
    
    # Ação e Recompensa são específicas (Mensalidade e LTV)
    scaler_acao = joblib.load("scaler_assinatura_acao.joblib")
    scaler_recompensa = joblib.load("scaler_assinatura_recompensa.joblib")
    
    # Carrega nomes das colunas para referência
    with open('colunas_estado_assinatura.json', 'r') as f:
        colunas_assinatura = json.load(f)
        
    print("✓ Scalers e metadados de assinatura carregados.")
    print(f"  Total de features no estado: {len(colunas_assinatura)}")
    
except FileNotFoundError as e:
    print(f"❌ ERRO: Arquivo não encontrado: {e.filename}")
    print("Execute o 'Generator_NEW.py' novamente para gerar os artefatos de assinatura.")


[1/5] Carregando ReplayBuffer de Assinatura de 'rl_assinatura_buffer.h5' ...
2025-11-21 09:46.47 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(36,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[()])
2025-11-21 09:46.47 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2025-11-21 09:46.47 [info     ] Action size has been automatically determined. action_size=1
✓ Buffer de Assinatura carregado com sucesso.
  # Transições totais: 50000

[2/5] Carregando Scalers de Assinatura...
✓ Scalers e metadados de assinatura carregados.
  Total de features no estado: 36


In [ ]:
# ============================================================================
# CÉLULA 3: Divisão Treino / Validação (Split Robusto) - CORRIGIDO
# ============================================================================
import numpy as np
from d3rlpy.dataset import Episode

print("\n[3/5] Dividindo dados em Treino e Teste...")

# Verifica se temos poucos episódios (ex: 1 episódio gigante)
if len(dataset.episodes) < 10:
    print(f"⚠️ Aviso: Encontrados apenas {len(dataset.episodes)} episódios.")
    print("   -> Dividindo o episódio gigante em sub-episódios menores para validação...")
    
    giant_ep = dataset.episodes[0]
    
    # Acessa arrays
    obs = giant_ep.observations
    act = giant_ep.actions
    rew = giant_ep.rewards
    
    # Reconstrói terminais (assumindo apenas o final como terminal)
    term = np.zeros((len(obs), 1), dtype=np.float32)
    # Tenta pegar atributo terminal ou terminated
    is_global_term = getattr(giant_ep, 'terminal', getattr(giant_ep, 'terminated', True))
    if is_global_term:
        term[-1] = 1.0
        
    total_len = len(obs)
    split_idx = int(total_len * 0.8)
    
    # Divide manualmente
    train_obs, test_obs = obs[:split_idx], obs[split_idx:]
    train_act, test_act = act[:split_idx], act[split_idx:]
    train_rew, test_rew = rew[:split_idx], rew[split_idx:]
    train_term, test_term = term[:split_idx], term[split_idx:]
    
    # Helper seguro (POSICIONAL para evitar erro de keyword)
    def create_ep(o, a, r, t):
        r = r.reshape(-1, 1) if r.ndim == 1 else r
        is_term = bool(t[-1]) if len(t) > 0 else False
        # (obs, act, rew, terminated)
        return Episode(o, a, r, is_term)

    train_episodes = [create_ep(train_obs, train_act, train_rew, train_term)]
    test_episodes = [create_ep(test_obs, test_act, test_rew, test_term)]
    
else:
    # Se tivermos muitos episódios, usa o split padrão
    train_episodes, test_episodes = train_test_split(dataset.episodes, test_size=0.2, random_state=42)

print(f"  Episódios de Treino: {len(train_episodes)}")
print(f"  Episódios de Teste:  {len(test_episodes)}")


[3/5] Dividindo dados em Treino e Teste...
  Episódios de Treino: 40000
  Episódios de Teste:  10000


In [4]:
# ============================================================================
# CÉLULA 4: Configuração do Agente CQL (Foco em LTV)
# ============================================================================
print("\n[4/5] Configurando Agente CQL (Assinatura/LTV)...")

# DIFERENÇA CHAVE: gamma=0.99
# Para assinatura, o longo prazo (LTV) é muito mais importante que o lucro imediato.
# Gamma mais alto faz o agente valorizar a retenção futura.

cql_config = CQLConfig(
    batch_size=256,
    gamma=0.99,               # <--- Foco em Longo Prazo (LTV)
    observation_scaler=None,  # Já escalado no Generator
    action_scaler=None,
    reward_scaler=None,
    alpha_learning_rate=1e-4,
    actor_learning_rate=1e-4,
    critic_learning_rate=3e-4,
    conservative_weight=5.0,  # Conservadorismo mantém-se alto
    q_func_factory=QRQFunctionFactory(n_quantiles=64)
)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
cql = cql_config.create(device=device)

print(f"✓ Agente CQL (LTV) configurado no dispositivo: {device}")


[4/5] Configurando Agente CQL (Assinatura/LTV)...
✓ Agente CQL (LTV) configurado no dispositivo: cpu


In [9]:
# ============================================================================
# CÉLULA 5: Treinamento com Governança (Loop Manual)
# ============================================================================
print("\n[5/5] Iniciando Treinamento de Assinatura...")

N_STEPS = 50000
STEPS_PER_EPOCH = 1000
PATIENCE = 20
BEST_MODEL_PATH = "modelo_rl_assinatura.pt" # Nome específico para este modelo

# Avaliador
avg_q_evaluator = AverageValueEstimationEvaluator(test_episodes)
evaluators = {'average_q': avg_q_evaluator}

# Recria buffer de treino para o fitter
train_buffer = ReplayBuffer(
    FIFOBuffer(limit=dataset.transition_count),
    episodes=train_episodes
)

best_score = -float('inf')
patience_counter = 0

# Loop de Treino
for epoch, metrics in cql.fitter(
    train_buffer,
    n_steps=N_STEPS,
    n_steps_per_epoch=STEPS_PER_EPOCH,
    evaluators=evaluators,
    experiment_name="cql_assinatura_run",
    with_timestamp=False,
    show_progress=True
):
    current_score = metrics.get('average_q')
    
    if current_score > best_score:
        best_score = current_score
        patience_counter = 0
        cql.save_model(BEST_MODEL_PATH)
        print(f"    [Epoch {epoch}] Novo recorde LTV! Score: {current_score:.4f}")
    else:
        patience_counter += 1
        print(f"    [Epoch {epoch}] Sem melhora. Score: {current_score:.4f} (Paciência: {patience_counter}/{PATIENCE})")
        
    if patience_counter >= PATIENCE:
        print(f"\n🛑 Early Stopping: Treino de assinatura encerrado.")
        break

if os.path.exists(BEST_MODEL_PATH):
    cql.load_model(BEST_MODEL_PATH)
    print(f"\n✓ Melhor modelo de Assinatura carregado de '{BEST_MODEL_PATH}'.")


[5/5] Iniciando Treinamento de Assinatura...
2025-11-21 10:53.57 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(36,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-11-21 10:53.57 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2025-11-21 10:53.57 [info     ] Action size has been automatically determined. action_size=1
2025-11-21 10:53.57 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(36,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=1)
2025-11-21 10:53.57 [warning  ] Skip building models since they're already built.
2025-11-21 10:53.

Epoch 1/50: 100%|██████████| 1000/1000 [05:33<00:00,  3.00it/s, critic_loss=-58.2, conservative_loss=-61, alpha=0.951, actor_loss=-0.0856, temp=0.955, temp_loss=1.39]


2025-11-21 10:59.45 [info     ] cql_assinatura_run: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.009842555284500122, 'time_algorithm_update': 0.32185865807533265, 'critic_loss': -58.21702280235291, 'conservative_loss': -60.996804996490475, 'alpha': 0.9509843505620956, 'actor_loss': -0.08032187248929404, 'temp': 0.9542893256545066, 'temp_loss': 1.3902101123332977, 'time_step': 0.3320035345554352, 'average_q': -0.18790360231772066} step=1000
2025-11-21 10:59.45 [info     ] Model parameters are saved to d3rlpy_logs\cql_assinatura_run\model_1000.d3
    [Epoch 1] Novo recorde LTV! Score: -0.1879


Epoch 2/50: 100%|██████████| 1000/1000 [04:41<00:00,  3.56it/s, critic_loss=-62, conservative_loss=-64.7, alpha=0.858, actor_loss=1.06, temp=0.885, temp_loss=0.764]  


2025-11-21 11:04.39 [info     ] cql_assinatura_run: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.00908719825744629, 'time_algorithm_update': 0.2705505242347717, 'critic_loss': -61.97115071105957, 'conservative_loss': -64.76444079208375, 'alpha': 0.8579332389235497, 'actor_loss': 1.06933987878263, 'temp': 0.8845283337235451, 'temp_loss': 0.7620396106541156, 'time_step': 0.2798525855541229, 'average_q': -1.0150455513812602} step=2000
2025-11-21 11:04.39 [info     ] Model parameters are saved to d3rlpy_logs\cql_assinatura_run\model_2000.d3
    [Epoch 2] Sem melhora. Score: -1.0150 (Paciência: 1/20)


...

2025-11-21 11:11.06 [info     ] cql_assinatura_run: epoch=3 step=3000 epoch=3 metrics={'time_sample_batch': 0.009180633783340454, 'time_algorithm_update': 0.36232124066352844, 'critic_loss': -67.94938455963134, 'conservative_loss': -74.39723878860474, 'alpha': 0.7720177013874054, 'actor_loss': 3.364053863286972, 'temp': 0.8300682639479637, 'temp_loss': 0.4650001800954342, 'time_step': 0.3717142481803894, 'average_q': -3.3496940682578833} step=3000
2025-11-21 11:11.07 [info     ] Model parameters are saved to d3rlpy_logs\cql_assinatura_run\model_3000.d3
    [Epoch 3] Sem melhora. Score: -3.3497 (Paciência: 2/20)


Epoch 4/50: 100%|██████████| 1000/1000 [07:54<00:00,  2.11it/s, critic_loss=-87, conservative_loss=-101, alpha=0.688, actor_loss=9.38, temp=0.79, temp_loss=0.22]    


2025-11-21 11:19.17 [info     ] cql_assinatura_run: epoch=4 step=4000 epoch=4 metrics={'time_sample_batch': 0.009501899242401123, 'time_algorithm_update': 0.4632861466407776, 'critic_loss': -87.1162483215332, 'conservative_loss': -101.60956675720215, 'alpha': 0.6871868042349816, 'actor_loss': 9.417908043146133, 'temp': 0.789750754058361, 'temp_loss': 0.2193962922496721, 'time_step': 0.4730088891983032, 'average_q': -9.578665587516129} step=4000
2025-11-21 11:19.18 [info     ] Model parameters are saved to d3rlpy_logs\cql_assinatura_run\model_4000.d3
    [Epoch 4] Sem melhora. Score: -9.5787 (Paciência: 3/20)


Epoch 5/50:  86%|████████▋ | 865/1000 [04:54<00:45,  2.94it/s, critic_loss=-116, conservative_loss=-141, alpha=0.611, actor_loss=19, temp=0.769, temp_loss=0.0539]  


KeyboardInterrupt: 

In [8]:
# ============================================================================
# CÉLULA 6: Avaliação Final e Métricas de Negócio - CORRIGIDA (Format Fix)
# ============================================================================
import os
import numpy as np
from d3rlpy.metrics import AverageValueEstimationEvaluator

print("\n" + "="*40)
print("AVALIAÇÃO DE ASSINATURA (LTV)")
print("="*40)

# 1. Construir a Rede Neural (Inicialização)
print("Construindo arquitetura do modelo...")
cql.build_with_dataset(dataset)

# 2. Carregar o melhor modelo salvo (Checkpoint)
if os.path.exists("modelo_rl_assinatura.pt"):
    cql.load_model("modelo_rl_assinatura.pt")
    print("✓ Melhor modelo ('modelo_rl_assinatura.pt') carregado com sucesso.")
else:
    print("⚠️ Aviso: 'modelo_rl_assinatura.pt' não encontrado. Usando modelo não treinado.")

# 3. Inferência de Exemplo
print(f"\n--- Exemplo de Inferência (Um Caso do Teste) ---")

try:
    sample_episode = test_episodes[0]
    sample_state = sample_episode.observations[0]

    # Garantir 2D (Batch Dimension)
    state_batch = sample_state.reshape(1, -1)

    # A. Prever Ação (Mensalidade Normalizada) - Retorna Array
    acao_norm_array = cql.predict(state_batch)[0]
    
    # Extrair valor escalar para exibição (Correção do Erro)
    acao_norm_valor = float(acao_norm_array) if isinstance(acao_norm_array, np.ndarray) else acao_norm_array

    # B. Converter para Real (Mensalidade em Dólares)
    mensalidade_real = scaler_acao.inverse_transform(acao_norm_array.reshape(1, -1))[0][0]

    # C. Calcular LTV Esperado (Risco)
    if hasattr(cql, "predict_value"):
        action_batch = acao_norm_array.reshape(1, -1)
        ltv_esperado_norm = cql.predict_value(state_batch, action_batch)[0]
        # Usa o scaler de recompensa específico de assinatura
        ltv_esperado_real = scaler_recompensa.inverse_transform(ltv_esperado_norm.reshape(1, -1))[0][0]
    else:
        ltv_esperado_real = 0.0

    print(f"  Estado (Shape): {sample_state.shape} (Inclui memória)")
    print(f"  Ação (Mensalidade Normalizada): {acao_norm_valor:.4f}")
    print(f"  ------------------------------------------------")
    print(f"  ✅ MENSALIDADE RECOMENDADA: ${mensalidade_real:.2f} / mês")
    print(f"  💰 LTV ESPERADO (Lifetime): ${ltv_esperado_real:.2f}")

    if mensalidade_real < 0:
        print("\n❌ ALERTA: Mensalidade negativa. Verifique custos de assinatura no Generator.")
    else:
        print("\n✅ SUCESSO: O modelo de assinatura está funcional!")

except Exception as e:
    print(f"\n❌ Erro durante a inferência: {e}")
    # Imprime o tipo para debug
    import traceback
    traceback.print_exc()


AVALIAÇÃO DE ASSINATURA (LTV)
Construindo arquitetura do modelo...
2025-11-21 09:56.44 [warning  ] Parameters will be reinitialized.
⚠️ Aviso: 'modelo_rl_assinatura.pt' não encontrado. Usando modelo não treinado.

--- Exemplo de Inferência (Um Caso do Teste) ---


C:\Users\MDF\AppData\Local\Temp\ipykernel_7608\1233218246.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  acao_norm_valor = float(acao_norm_array) if isinstance(acao_norm_array, np.ndarray) else acao_norm_array


  Estado (Shape): (36,) (Inclui memória)
  Ação (Mensalidade Normalizada): -0.1181
  ------------------------------------------------
  ✅ MENSALIDADE RECOMENDADA: $91.04 / mês
  💰 LTV ESPERADO (Lifetime): $486.73

✅ SUCESSO: O modelo de assinatura está funcional!


In [5]:
# --- Etapa 4: Formatando dados de TREINO e TESTE para o d3rlpy (CORRIGIDO v2) ---
print("\n--- Etapa 4: Formatando dados de TREINO e TESTE para o d3rlpy ---")

# Inicializa as variáveis globais fora do IF-block
state_features = []
category_state_features = []
observation_cols_map = []
train_replay_buffer = None
test_replay_buffer = None
train_buffer_creation_successful = False
test_buffer_creation_successful = False

if not df_train_rl.empty and not df_original.empty:

    # 4.1. Definir Features de Estado
    all_original_cols = set(df_original.columns)
    cols_to_exclude = {'Churn', 'MonthlyCharges', 'TotalCharges'}
    state_features = sorted(list(all_original_cols - cols_to_exclude))
    category_state_features = sorted(list(set(category_cols) & set(state_features)))

    print(f"Features de Estado (State) identificadas ({len(state_features)}): {state_features}")
    print(f"Features Categóricas (para One-Hot) ({len(category_state_features)}): {category_state_features}")

    # 4.2. Função para processar DataFrame (Treino ou Teste)
    # --- CORREÇÃO AQUI: Adicionado 'is_inference=False' ---
    def process_df_for_d3rlpy(df, state_cols, category_cols_in_state, observation_cols_map=None, is_inference=False):
        """Converte um DataFrame para o formato de arrays do d3rlpy."""

        df_processed = df.copy()

        # 1. One-Hot Encoding
        df_onehot = pd.get_dummies(df_processed[state_cols],
                                   columns=category_cols_in_state,
                                   dummy_na=False)

        # 2. Alinhamento de Colunas
        if observation_cols_map is None:
            observation_cols_map = sorted(list(df_onehot.columns))
        else:
            df_onehot = df_onehot.reindex(columns=observation_cols_map, fill_value=0)

        # 3. Criar Arrays NumPy
        observations = df_onehot[observation_cols_map].values.astype(np.float32)

        # --- CORREÇÃO AQUI: Só processa ações/recompensas se NÃO for inferência ---
        if not is_inference:
            actions = df_processed[['Price_Action']].values.astype(np.float32)
            rewards = df_processed['Simulated_Profit_Reward'].values.astype(np.float32).reshape(-1, 1)
            terminated = np.ones_like(rewards, dtype=np.float32).reshape(-1, 1)
            return observations, actions, rewards, terminated, observation_cols_map
        else:
            # Se for inferência, só precisamos das observações
            return observations, None, None, None, observation_cols_map
    # -----------------------------------------------------------------

    # 4.3. Função para criar ReplayBuffer
    def create_replay_buffer(observations, actions, rewards, terminated):
        """Cria um ReplayBuffer do d3rlpy a partir de arrays NumPy."""
        try:
            buffer_size = len(rewards)
            episodes = []
            for i in range(buffer_size):
                episode = Episode(
                    observations=observations[i:i+1],
                    actions=actions[i:i+1],
                    rewards=rewards[i:i+1],
                    terminated=terminated[i:i+1]
                )
                episodes.append(episode)

            replay_buffer = ReplayBuffer(
                buffer=FIFOBuffer(limit=buffer_size),
                episodes=episodes,
                cache_size=16
            )
            return replay_buffer
        except Exception as e:
            print(f"!!! ERRO CRÍTICO ao criar Buffer: {e}")
            return None

    # 4.4. Processar TREINO (is_inference continua False por defeito)
    print("\n--- Processando Conjunto de Treino ---")
    train_obs, train_act, train_rew, train_term, observation_cols_map = \
        process_df_for_d3rlpy(df_train_rl, state_features, category_state_features, observation_cols_map=None)

    print(f"Número de colunas de observação (features) após one-hot: {len(observation_cols_map)}")

    train_replay_buffer = create_replay_buffer(train_obs, train_act, train_rew, train_term)
    if train_replay_buffer:
        print(f"SUCESSO: ReplayBuffer de TREINO pronto com {len(train_replay_buffer.buffer)} transições.")
        train_buffer_creation_successful = True

    # 4.5. Processar TESTE (is_inference continua False por defeito)
    print("\n--- Processando Conjunto de Teste ---")
    test_obs, test_act, test_rew, test_term, _ = \
        process_df_for_d3rlpy(df_test_rl, state_features, category_state_features, observation_cols_map=observation_cols_map)

    test_replay_buffer = create_replay_buffer(test_obs, test_act, test_rew, test_term)
    if test_replay_buffer:
        print(f"SUCESSO: ReplayBuffer de TESTE pronto com {len(test_replay_buffer.buffer)} transições.")
        test_buffer_creation_successful = True

else:
    print("PULANDO Etapa 4: Datasets de treino/teste estão vazios.")


--- Etapa 4: Formatando dados de TREINO e TESTE para o d3rlpy ---


NameError: name 'df_train_rl' is not defined

In [ ]:
# --- Etapa 5: Configurando, Construindo e Treinando o Agente (CORRIGIDO v4) ---
print("\n--- Etapa 5: Configurando, Construindo e Treinando o Agente ---")

model_built_successfully = False
agent_trained_successfully = False
cql_pricer = None # O nome do agente

if train_buffer_creation_successful:
    try:
        print("Configurando o agente CQL com QRQFunctionFactory (para Ações Contínuas)...")

        # 5.1. Configurar o Agente
        cql_config = CQLConfig(
            q_func_factory=QRQFunctionFactory(n_quantiles=64),
            batch_size=256,
            n_action_samples=10,
            alpha_learning_rate=1e-4,
            conservative_weight=5.0
        )

        # 5.2. Criar o Agente
        device_to_use_str = "cuda" if torch.cuda.is_available() else "cpu"
        cql_pricer = cql_config.create(device=device_to_use_str)
        print(f"Agente criado e rodando em: {cql_pricer._device}")

        # 5.3. Construir o Agente com os dados
        print("Construindo o agente com as assinaturas do ReplayBuffer de TREINO...")
        cql_pricer.build_with_dataset(train_replay_buffer)
        model_built_successfully = True
        print("Agente construído com sucesso.")

        # 5.4. Treinar o Agente (Offline)
        N_TRAINING_EPOCHS = 10
        N_STEPS_PER_EPOCH = 100

        print(f"Iniciando treinamento offline por {N_TRAINING_EPOCHS} épocas ({N_STEPS_PER_EPOCH} steps/epoch)...")

        # --- CORREÇÃO AQUI ---
        # Removidos 'scorers' e 'eval_dataset' para evitar o TypeError,
        # alinhando com o notebook de referência (código_final_RL_OFF).

        cql_pricer.fit(
            train_replay_buffer,
            n_steps=N_TRAINING_EPOCHS * N_STEPS_PER_EPOCH,
            n_steps_per_epoch=N_STEPS_PER_EPOCH
        )

        agent_trained_successfully = True
        print("\n--- Treinamento Concluído com Sucesso ---")

    except Exception as e:
        print(f"\n!!! ERRO CRÍTICO durante a Etapa 5 (Construção/Treinamento): {e} !!!")
        import traceback
        traceback.print_exc()

else:
    print("Aviso: Treinamento PULADO. 'train_replay_buffer' não foi criado.")


--- Etapa 5: Configurando, Construindo e Treinando o Agente ---
Configurando o agente CQL com QRQFunctionFactory (para Ações Contínuas)...
Agente criado e rodando em: cpu
Construindo o agente com as assinaturas do ReplayBuffer de TREINO...
Agente construído com sucesso.
Iniciando treinamento offline por 10 épocas (100 steps/epoch)...
2025-11-05 16:27.34 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(42,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=1)
2025-11-05 16:27.34 [warning  ] Skip building models since they're already built.
2025-11-05 16:27.34 [info     ] Directory is created at d3rlpy_logs/CQL_20251105162734
2025-11-05 16:27.34 [info     ] Parameters                     params={'observation_shape': [42], 'action_size': 1, 'config': {'type': 'cql

Epoch 1/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-11-05 16:28.09 [info     ] CQL_20251105162734: epoch=1 step=100 epoch=1 metrics={'time_sample_batch': 0.008755757808685302, 'time_algorithm_update': 0.3387613415718079, 'critic_loss': 87.24948081970214, 'conservative_loss': -70.18566291809083, 'alpha': 0.994828377366066, 'actor_loss': -7.992976068854332, 'temp': 0.9967465716600418, 'temp_loss': 1.408757402896881, 'time_step': 0.3476544260978699} step=100
2025-11-05 16:28.09 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_100.d3


Epoch 2/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-11-05 16:28.43 [info     ] CQL_20251105162734: epoch=2 step=200 epoch=2 metrics={'time_sample_batch': 0.009012703895568847, 'time_algorithm_update': 0.3325561237335205, 'critic_loss': -135.73632007598877, 'conservative_loss': -151.86371910095215, 'alpha': 0.9821200197935105, 'actor_loss': -1.0730998655594886, 'temp': 0.986779014468193, 'temp_loss': 1.5636797916889191, 'time_step': 0.3417008757591248} step=200
2025-11-05 16:28.43 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_200.d3


...

2025-11-05 16:34.05 [info     ] CQL_20251105162734: epoch=8 step=800 epoch=8 metrics={'time_sample_batch': 0.009103341102600098, 'time_algorithm_update': 0.9102189230918885, 'critic_loss': -6006.136733398437, 'conservative_loss': -7691.232084960938, 'alpha': 0.8662542647123337, 'actor_loss': 850.3849615478516, 'temp': 1.0002096778154372, 'temp_loss': -2.9471511054039, 'time_step': 0.919463529586792} step=800
2025-11-05 16:34.05 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_800.d3


Epoch 9/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-11-05 16:35.19 [info     ] CQL_20251105162734: epoch=9 step=900 epoch=9 metrics={'time_sample_batch': 0.009109461307525634, 'time_algorithm_update': 0.7249660420417786, 'critic_loss': -10101.297177734376, 'conservative_loss': -12957.51720703125, 'alpha': 0.8443763309717178, 'actor_loss': 1472.6126647949218, 'temp': 1.0179425823688506, 'temp_loss': -3.58377925157547, 'time_step': 0.734211950302124} step=900
2025-11-05 16:35.19 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_900.d3


Epoch 10/10:   0%|          | 0/100 [00:00<?, ?it/s]

2025-11-05 16:36.22 [info     ] CQL_20251105162734: epoch=10 step=1000 epoch=10 metrics={'time_sample_batch': 0.009180173873901368, 'time_algorithm_update': 0.6224578094482421, 'critic_loss': -15580.564853515625, 'conservative_loss': -20202.903984375, 'alpha': 0.8236886262893677, 'actor_loss': 2343.871702880859, 'temp': 1.0354898595809936, 'temp_loss': -4.040697162151337, 'time_step': 0.6317691874504089} step=1000
2025-11-05 16:36.22 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20251105162734/model_1000.d3

--- Treinamento Concluído com Sucesso ---


In [ ]:
# --- Sub-Etapa 5.5: Definição das Funções para Análise de Risco ---
print("\n--- Sub-Etapa 5.5: Definindo funções para análise de risco (VaR, CVaR) ---")

def get_quantile_values(agent, observation_np, action_np):
    """Obtém os valores dos quantis previstos pela Q-Function (QR) do agente."""
    if not (agent and model_built_successfully and agent_trained_successfully):
        print("[Debug get_quantile_values]: Agente não está pronto.")
        return None

    try:
        device = agent._device
        obs_tensor = torch.tensor(observation_np, dtype=torch.float32, device=device).reshape(1, -1)
        act_tensor = torch.tensor(action_np, dtype=torch.float32, device=device).reshape(1, -1)

        if not (hasattr(agent, 'impl') and hasattr(agent.impl, '_q_func_forwarder') and
                agent.impl._q_func_forwarder._forwarders):
            print("[Debug get_quantile_values]: Estrutura do agente (impl._q_func_forwarder) não encontrada.")
            return None

        q_func_forwarder = agent.impl._q_func_forwarder._forwarders[0]
        critic_network = q_func_forwarder._q_func
        critic_network.eval() # Modo de avaliação

        with torch.no_grad():
            q_output = critic_network(obs_tensor, act_tensor)

        quantile_tensor = q_output.quantiles
        quantile_values_np = quantile_tensor.cpu().numpy().squeeze()

        if isinstance(quantile_values_np, np.ndarray) and quantile_values_np.ndim == 1:
             return quantile_values_np
        else:
             print("[Debug get_quantile_values]: A saída dos quantis não é um array 1D.")
             return None

    except Exception as e:
        print(f"!!! Erro inesperado em get_quantile_values: {e} !!!")
        import traceback
        traceback.print_exc()
        return None

def calculate_var(distribution_values, alpha=0.05):
    """Calcula o Valor em Risco (VaR)."""
    if distribution_values is None or not isinstance(distribution_values, np.ndarray) or distribution_values.size == 0:
        return np.nan
    sorted_returns = np.sort(distribution_values)
    var_index = int(alpha * len(sorted_returns))
    var_index = max(0, min(var_index, len(sorted_returns) - 1))
    return sorted_returns[var_index]

def calculate_cvar(distribution_values, alpha=0.05):
    """Calcula o Valor Condicional em Risco (CVaR)."""
    if distribution_values is None or not isinstance(distribution_values, np.ndarray) or distribution_values.size == 0:
        return np.nan
    var_value = calculate_var(distribution_values, alpha)
    if np.isnan(var_value): return np.nan
    worse_than_var = distribution_values[distribution_values <= var_value]
    return np.mean(worse_than_var) if worse_than_var.size > 0 else np.nan

print("Funções de análise de risco definidas.")


--- Sub-Etapa 5.5: Definindo funções para análise de risco (VaR, CVaR) ---
Funções de análise de risco definidas.


In [ ]:
# --- Sub-Etapa 5.6: Avaliação de Risco da Política (em dados de TESTE) ---
print("\n--- Sub-Etapa 5.6: Avaliação de Risco da Política (em dados de TESTE) ---")

if agent_trained_successfully and test_buffer_creation_successful:
    SAMPLE_SIZE = min(1000, len(test_replay_buffer.buffer))
    ALPHA_RISK = 0.05

    sample_indices = np.random.choice(len(test_replay_buffer.buffer), SAMPLE_SIZE, replace=False)

    results = []
    actual_rewards_eval = []
    predicted_mean_profits_eval = []

    print(f"Processando {SAMPLE_SIZE} transições aleatórias do ReplayBuffer de TESTE...")

    for index in sample_indices:
        try:
            transition = test_replay_buffer.episodes[index]
            obs = np.asarray(transition.observations[0])
            actual_reward = np.asarray(transition.rewards[0])[0]

            # Prever Ação Ótima
            optimal_action = cql_pricer.predict(obs.reshape(1, -1))[0]

            # Obter Distribuição de Quantis
            predicted_quantiles = get_quantile_values(cql_pricer, obs, optimal_action)

            if predicted_quantiles is not None and predicted_quantiles.size > 0:
                mean_profit = np.mean(predicted_quantiles)
                var_value = calculate_var(predicted_quantiles, alpha=ALPHA_RISK)
                cvar_value = calculate_cvar(predicted_quantiles, alpha=ALPHA_RISK)

                results.append({
                    'predicted_price': optimal_action[0],
                    'mean_sim_profit_agent': mean_profit,
                    'VaR_5': var_value,
                    'CVaR_5': cvar_value
                })

                actual_rewards_eval.append(actual_reward)
                predicted_mean_profits_eval.append(mean_profit)
        except Exception as e:
            print(f"Erro ao processar amostra {index}: {e}")

    # 5.7. Apresentar Resultados
    if results:
        results_df = pd.DataFrame(results)

        print(f"\n--- Resultados Agregados da Avaliação de Risco (Buffer de TESTE) ---")
        print(f" Nível Alpha: {ALPHA_RISK*100:.1f}% | Amostras: {len(results_df)}")
        print("-" * 70)
        print(f" Preço Médio Recomendado pela Política:        ${results_df['predicted_price'].mean():,.2f}")
        print(f" Lucro Simulado Médio PREVISTO pelo Agente:    ${results_df['mean_sim_profit_agent'].mean():,.2f}")
        print(f" VaR (5%) Médio Simulado (Previsto Agente): ${results_df['VaR_5'].mean(skipna=True):,.2f}")
        print(f" CVaR (5%) Médio Simulado (Previsto Agente):${results_df['CVaR_5'].mean(skipna=True):,.2f}")
        print("-" * 70)

        if actual_rewards_eval:
            mae_eval = mean_absolute_error(actual_rewards_eval, predicted_mean_profits_eval)
            avg_actual_profit_buffer = np.mean(actual_rewards_eval)
            print(f"\n--- Métricas de PRECISÃO da Previsão Média do Agente (vs Buffer TESTE) ---")
            print(f" Lucro Simulado Médio REAL no Buffer Avaliado: ${avg_actual_profit_buffer:,.2f}")
            print(f" MAE (Erro Médio Absoluto da Previsão Média):  ${mae_eval:,.2f}")
            print("-" * 70)
    else:
        print("Nenhum resultado de avaliação de risco foi gerado.")

else:
    print("Avaliação de Risco PULADA. Agente ou buffer de teste não estão prontos.")


--- Sub-Etapa 5.6: Avaliação de Risco da Política (em dados de TESTE) ---
Processando 1000 transições aleatórias do ReplayBuffer de TESTE...

--- Resultados Agregados da Avaliação de Risco (Buffer de TESTE) ---
 Nível Alpha: 5.0% | Amostras: 1000
----------------------------------------------------------------------
 Preço Médio Recomendado pela Política:        $1.00
 Lucro Simulado Médio PREVISTO pelo Agente:    $-2,649.53
 VaR (5%) Médio Simulado (Previsto Agente): $-7,438.05
 CVaR (5%) Médio Simulado (Previsto Agente):$-8,913.42
----------------------------------------------------------------------

--- Métricas de PRECISÃO da Previsão Média do Agente (vs Buffer TESTE) ---
 Lucro Simulado Médio REAL no Buffer Avaliado: $10.12
 MAE (Erro Médio Absoluto da Previsão Média):  $2,659.65
----------------------------------------------------------------------


In [ ]:
# --- Etapa 6: Gerando Recomendações de Preço Específicas (CORRIGIDO) ---
print("\n--- Etapa 6: Gerando Recomendações de Preço Específicas ---")

# Variáveis globais necessárias (da Etapa 4)
# state_features, observation_cols_map, category_state_features

def get_price_recommendation(**scenario_kwargs):
    """
    Gera uma recomendação de preço para um cenário de cliente.
    As kwargs devem corresponder às 'state_features' do dataset de churn.
    """

    if not (agent_trained_successfully and 'observation_cols_map' in globals() and observation_cols_map):
        print("Erro: Agente não treinado ou 'observation_cols_map' não definido.")
        return

    # 1. Criar DataFrame do cenário
    default_scenario = {}
    for col in state_features:
        if col in df_original.columns:
            if df_original[col].dtype == 'object':
                default_scenario[col] = df_original[col].mode()[0]
            else:
                default_scenario[col] = df_original[col].median()
        else:
            default_scenario[col] = 0

    for key, value in scenario_kwargs.items():
        if key in default_scenario:
            default_scenario[key] = value

    scenario_df = pd.DataFrame([default_scenario])

    # 2. Processar o cenário (One-hot e Alinhamento)
    try:
        # --- CORREÇÃO AQUI: Passa 'is_inference=True' ---
        scenario_obs, _, _, _, _ = process_df_for_d3rlpy(
            scenario_df,
            state_features,
            category_state_features,
            observation_cols_map=observation_cols_map,
            is_inference=True  # Diz à função para não procurar 'Price_Action'
        )
        observation = scenario_obs.reshape(1, -1)

    except Exception as e:
        print(f"Erro ao processar o cenário: {e}")
        return

    # 3. Prever Ação (Preço) e Risco
    try:
        recommended_price = cql_pricer.predict(observation)[0]

        print(f"\nCenário:")
        print(json.dumps(scenario_kwargs, indent=2))
        print(f"  => Preço Recomendado: ${recommended_price[0]:.2f}")

        # 4. Calcular Risco (VaR/CVaR)
        predicted_quantiles = get_quantile_values(cql_pricer, observation, recommended_price)

        if predicted_quantiles is not None and predicted_quantiles.size > 0:
            mean_profit = np.mean(predicted_quantiles)
            var_5 = calculate_var(predicted_quantiles, alpha=0.05)
            cvar_5 = calculate_cvar(predicted_quantiles, alpha=0.05)
            print(f"     Lucro Médio Previsto: ${mean_profit:.2f}")
            print(f"     VaR (5%): ${var_5:.2f} | CVaR (5%): ${cvar_5:.2f}")
        else:
            print("     (Não foi possível calcular VaR/CVaR para esta recomendação)")
        print("-" * 30)

    except Exception as e:
        print(f"Erro durante a predição para o cenário: {e}")

# --- Exemplos de Recomendação ---
if agent_trained_successfully:
    print("Gerando recomendações de exemplo...")
    get_price_recommendation(
        SubscriptionType='Standard',
        Gender='Female',
        DeviceRegistered='Tablet',
        ViewingHoursPerWeek=40
    )

    get_price_recommendation(
        SubscriptionType='Basic',
        Gender='Male',
        DeviceRegistered='Mobile',
        AccountAge=5 # Baixa idade
    )

    get_price_recommendation(
        SubscriptionType='Premium',
        ViewingHoursPerWeek=50,
        ContentDownloadsPerMonth=30,
        UserRating=4.5
    )
else:
    print("Recomendações PULADAS. Agente não treinado.")


--- Etapa 6: Gerando Recomendações de Preço Específicas ---
Gerando recomendações de exemplo...

Cenário:
{
  "SubscriptionType": "Standard",
  "Gender": "Female",
  "DeviceRegistered": "Tablet",
  "ViewingHoursPerWeek": 40
}
  => Preço Recomendado: $1.00
     Lucro Médio Previsto: $-2941.21
     VaR (5%): $-8269.86 | CVaR (5%): $-9912.32
------------------------------

Cenário:
{
  "SubscriptionType": "Basic",
  "Gender": "Male",
  "DeviceRegistered": "Mobile",
  "AccountAge": 5
}
  => Preço Recomendado: $1.00
     Lucro Médio Previsto: $-2394.98
     VaR (5%): $-6443.26 | CVaR (5%): $-7693.88
------------------------------

Cenário:
{
  "SubscriptionType": "Premium",
  "ViewingHoursPerWeek": 50,
  "ContentDownloadsPerMonth": 30,
  "UserRating": 4.5
}
  => Preço Recomendado: $1.00
     Lucro Médio Previsto: $-3246.75
     VaR (5%): $-9107.43 | CVaR (5%): $-10914.38
------------------------------


In [ ]:
# --- Etapa 9: Comparação com Aprendizado Supervisionado (SL-Regressor) ---
print("\n--- Etapa 9: Iniciando Comparação com Supervised Learning (LGBM) ---")

if not df_rl_training.empty and 'observation_cols_map' in globals():

    print("Formatando dados para o modelo de SL...")

    # Re-processa o df_train_rl
    df_train_sl_X, _, _, _, _ = process_df_for_d3rlpy(
        df_train_rl, state_features, category_state_features, observation_cols_map
    )
    df_train_sl_X = pd.DataFrame(df_train_sl_X, columns=observation_cols_map)
    df_train_sl_X['Price_Action'] = df_train_rl['Price_Action']

    y_train_sl = df_train_rl['Simulated_Profit_Reward']

    # Repete para o Teste
    df_test_sl_X, _, _, _, _ = process_df_for_d3rlpy(
        df_test_rl, state_features, category_state_features, observation_cols_map
    )
    df_test_sl_X = pd.DataFrame(df_test_sl_X, columns=observation_cols_map)
    df_test_sl_X['Price_Action'] = df_test_rl['Price_Action']

    y_test_sl = df_test_rl['Simulated_Profit_Reward']

    print(f"Dados SL divididos em {len(df_train_sl_X)} para treino e {len(df_test_sl_X)} para teste.")

    # 9.2. Treinar o Regressor LightGBM
    print("\nTreinando o modelo LightGBM para prever o lucro...")

    lgbm_regressor = lgb.LGBMRegressor(random_state=42, n_estimators=200)
    lgbm_regressor.fit(df_train_sl_X, y_train_sl)
    print("Modelo SL (Regressor) treinado com sucesso!")

    # 9.3. Avaliar a PRECISÃO do Regressor
    print("\nAvaliando a precisão do modelo SL no conjunto de teste...")
    y_pred_sl = lgbm_regressor.predict(df_test_sl_X)

    mae_sl = mean_absolute_error(y_test_sl, y_pred_sl)
    r2_sl = r2_score(y_test_sl, y_pred_sl)

    print(f"----------- MÉTRICAS DE PRECISÃO (Regressão do Lucro) -----------")
    print(f"Erro Médio Absoluto (MAE): ${mae_sl:,.2f}")
    print(f"R-quadrado (R²): {r2_sl:.2%}")
    print("----------------------------------------------------------------")
    print("Nota: Isto mede o quão bem o LGBM 'decorou' a função de simulação.")

else:
    print("PULANDO Etapa 9: Dataset 'df_rl_training' está vazio ou mapa de colunas não foi criado.")


--- Etapa 9: Iniciando Comparação com Supervised Learning (LGBM) ---
Formatando dados para o modelo de SL...
Dados SL divididos em 38520 para treino e 9630 para teste.

Treinando o modelo LightGBM para prever o lucro...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011862 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1294
[LightGBM] [Info] Number of data points in the train set: 38520, number of used features: 43
[LightGBM] [Info] Start training from score 9.997657
Modelo SL (Regressor) treinado com sucesso!

Avaliando a precisão do modelo SL no conjunto de teste...
----------- MÉTRICAS DE PRECISÃO (Regressão do Lucro) -----------
Erro Médio Absoluto (MAE): $0.01
R-quadrado (R²): 100.00%
----------------------------------------------------------------
N

In [ ]:
# --- Etapa 10: Salvando o Modelo e Componentes ---
print("\n--- Etapa 10: Salvando o Modelo e Componentes ---")

if agent_trained_successfully:
    try:
        cql_pricer.save_model('modelo_rl_churn_pricer.pt')
        print("Modelo salvo como 'modelo_rl_churn_pricer.pt'")

        with open('colunas_observacao_churn.json', 'w') as f:
            json.dump(observation_cols_map, f)
        print("Colunas de observação salvas como 'colunas_observacao_churn.json'")

        with open('config_tiers_churn.json', 'w') as f:
            json.dump(product_tiers, f)
        print("Configuração de Tiers salva como 'config_tiers_churn.json'")

    except Exception as e:
        print(f"Erro ao salvar arquivos: {e}")
else:
    print("Salvamento PULADO. Agente não foi treinado.")

print("\n--- FIM DO SCRIPT ---")


--- Etapa 10: Salvando o Modelo e Componentes ---
Modelo salvo como 'modelo_rl_churn_pricer.pt'
Colunas de observação salvas como 'colunas_observacao_churn.json'
Configuração de Tiers salva como 'config_tiers_churn.json'

--- FIM DO SCRIPT ---
